In [1]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm 
import statistics 


In [2]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities.csv")
sim_df = pd.read_csv(f"{results_dir}/context_response_similarity_scores.csv")

cities_df = pd.read_csv("../../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

In [3]:
results_df.head()

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ..."


In [4]:
# average response length

avg_len = np.mean(results_df['response'].apply(len))
avg_len_sar = np.mean(results_df['response_sustainable'].apply(len))

In [33]:
def is_short(text):
    if len(text) <= 1000:
        return True
    return False 

short_results = results_df[(results_df['response'].apply(is_short)) & (results_df['response_sustainable'].apply(is_short))]

In [32]:
short_results

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable
86,llama3point1-instruct,prompt_46_gemini-ui,['Sibiu'],"I recommend Sibiu, Romania, because it offers...",['Sibiu'],I recommend Sibiu. I recommend Sibiu because ...
103,llama3point1-instruct,prompt_4_gpt-4o-mini,['Vienna'],"I recommend Vienna, Austria for winter travel...",['Vienna'],I recommend Vienna for Christmas markets beca...
207,mistral-instruct,prompt_58_gemini-ui,['Milan'],1. I recommend Milan for your business trip in...,"['Milan', 'Hamburg']",1. I recommend Milan in Italy.\nMilan is a cit...
208,mistral-instruct,prompt_22_gemini-ui,"['Kaunas', 'Plovdiv', 'Nalchik', 'Erzurum']",1. Kaunas\n2. Kaunas\n3. Kaunas\n4. Kaunas\n5....,"['Kaunas', 'Kirov', 'Plovdiv', 'Nalchik', 'Erz...",1. The city of Kaunas in Lithuania has a susta...
209,mistral-instruct,prompt_7_gemini-ui,['Thessaloniki'],1. I recommend Thessaloniki in Greece. This ci...,['Arkhangelsk'],"1. I recommend Arkhangelsk, Russia. The city i..."
224,mistral-instruct,prompt_24_gemini-ui,['Maastricht'],1. I recommend Maastricht in the Netherlands. ...,['Nalchik'],"1. I recommend Nalchik, Russia.\n\nNalchik is ..."
229,mistral-instruct,prompt_10_gpt-4o-mini,['Arkhangelsk'],"1. I recommend Arkhangelsk, Russia. Arkhangels...",['Arkhangelsk'],"1. I recommend Arkhangelsk, Russia. Arkhangels..."
243,mistral-instruct,prompt_39_gemini-1.5-pro-001,"['Tallinn', 'Sivas', 'Klagenfurt', 'Pamplona',...",1. Tallinn\n2. Tartu\n3. Pärnu\n4. Vilnius\n5....,"['Tallinn', 'Sivas', 'Pamplona', 'Batman', 'Kl...",1. Tallinn in Estonia\n2. Sivas in Turkey\n3. ...
254,mistral-instruct,prompt_4_gemini-1.5-pro-001,['Kaunas'],"1. Based on the information provided, it seems...",['Simferopol'],"1. I recommend Simferopol, Ukraine as the best..."
257,mistral-instruct,prompt_37_gemini-ui,"['Budapest', 'Braga', 'Cheboksary', 'Rome']",1. Budapest\n2. Braga\n3. Cheboksary\n4. Baia ...,[],"1. I'm sorry, I didn't understand the question..."


In [31]:
df = short_results[(short_results['model'] == 'mistral-instruct') & (short_results['prompt_id'] == "prompt_24_gemini-ui")]
print(df['prompt_id'].item())

prompt_24_gemini-ui


In [29]:
print(df['response'].item())

1. I recommend Maastricht in the Netherlands. Maastricht is a beautiful city with stunning scenery and opportunities to meet other travelers. The city has many hiking trails and is surrounded by beautiful countryside. It is also home to a vibrant cultural scene and has a rich history. Visitors can explore the city on foot or by bike, and there are many guided tours available to help them discover the best sights and experiences. Maastricht is a great choice for solo travelers who are looking for a challenging and rewarding hiking experience.


In [30]:
print(df['response_sustainable'].item())

1. I recommend Nalchik, Russia.

Nalchik is a beautiful city located in the stunning Caucasus Mountains. It offers a range of outdoor activities, including hiking, mountain biking, and skiing. The city is also home to several cultural and historical sites, such as the Nalchik Kremlin and the Church of the Holy Mother of God. Overall, Nalchik is an excellent destination for travelers looking for a challenging hike, stunning scenery, and opportunities to meet other travelers.


In [34]:
l_df = short_results[(short_results['model'] == "llama3point1-instruct") & (short_results['prompt_id'] == "prompt_46_gemini-ui")]

In [38]:
print(l_df['response'].item())

 I recommend Sibiu, Romania, because it offers a mix of historic city exploration, art, and fine dining, which aligns perfectly with your preferences. The city's old center has many options for traditional full meals, and the Il Padrino pizzeria is a great choice. Additionally, the city has a lively atmosphere, and you can find many souvenir and craft shops, including those selling German language books. The ideal month to visit Sibiu is March, as the weather is mild and pleasant, making it perfect for exploring the city's historic sites and enjoying its cultural events.


In [39]:
print(l_df['response_sustainable'].item())

 I recommend Sibiu. I recommend Sibiu because it offers a great combination of historic architecture, cultural attractions, and a lively atmosphere, making it an ideal destination for a romantic getaway in March. The city's picturesque old town, with its well-preserved buildings and charming squares, is a perfect setting for a romantic stroll. Additionally, Sibiu's rich cultural scene, including its numerous museums and art galleries, will keep you and your partner engaged and entertained. With its pleasant spring weather in March, Sibiu is an excellent choice for a romantic escape.
